






### Instruction
Spam filtering is a beginner’s example of the document classification task which involves classifying an email as spam or non-spam (a.k.a. ham) mail. An email dataset will be provided. We will use the following steps to build this application:
1) Preparing the text data
2) Creating a word dictionary
3) Feature extraction
4) Training the classifier
5) Checking the results on the test set

### Preparing the text data
The data-set used here, is split into a training set and a test set containing 702 mails and 260 mails respectively, divided equally between spam and ham mails. You will easily recognize spam mails as it contains `spmsg` in its filename.

In any text mining problem, text cleaning is the first step where we remove those words from the document which may not contribute to the information we want to extract. Emails may contain a lot of undesirable characters like punctuation marks, stop words, digits, etc which may not be helpful in detecting the spam email. The emails in Ling-spam corpus have been already preprocessed in the following ways:

1. **Removal of stop words** – Stop words like “and”, “the”, “of”, etc are very common in all English sentences and are not very meaningful in deciding spam or legitimate status, so these words have been removed from the emails.

2. **Lemmatization** – It is the process of grouping together the different inflected forms of a word so they can be analyzed as a single item. For example, “include”, “includes,” and “included” would all be represented as “include”. The context of the sentence is also preserved in lemmatization as opposed to stemming (another buzz word in text mining which does not consider meaning of the sentence)

We still need to remove the non-words like punctuation marks or special characters from the mail documents. There are several ways to do it. Here, we will remove such words after creating a dictionary, which is a very convenient method to do so since when you have a dictionary; you need to remove every such word only once.

### Creating word dictionary
We will only perform text analytics on the content to detect the spam mails. As the first step, we need to create a dictionary of words and their frequency. For this task, a training set of 700 mails is utilized. This python function will create the dictionary for you.
```Python
def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir (train_dir)]
    all_words = []
    for mail in emails:
        with open (mail) as m:
            for i,line in enumerate (m) :
                if i == 2:
                    words = line.split()
                    all_words += words
    dictionary = Counter(all_words)
    # Paste code for non-word removal here
    
    return dictionary
```

Once the dictionary is created we can add just a few lines of code written below to the above function to remove non-words. Absurd single characters in the dictionary which are irrelevant here are also removed. Do not forget to insert the below code in the function of make_Dictionary:
```python
list_to_remove = list(dictionary.keys())
for item in list_to_remove:
    if item.isalpha() == False:
        del dictionary[item]
    elif len(item) == 1:
        del dictionary[item]
dictionary = dictionary.most_common(3000)
```

In [1]:
import os
import numpy as np
from collections import Counter
def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir (train_dir)]
    all_words = []
    for mail in emails:
        with open (mail) as m:
            for i,line in enumerate (m) :
                if i == 2:
                    words = line.split()
                    all_words += words
    dictionary = Counter(all_words)
    list_to_remove = list(dictionary.keys())
    for item in list_to_remove:
        if item.isalpha() == False:
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
    dictionary = dictionary.most_common(3000)
    # Paste code for non-word removal here
    return dictionary

The dictionary can be seen by the command “print dictionary”. You may find some absurd word counts to be high but don’t worry, it’s just a dictionary and you always have a chance to improve it later. If you use the provided dataset, make sure your dictionary has some of the entries given below as most frequent words. Here 3000 most frequently used words are chosen in the dictionary.

In [2]:
# To show the most frequent words in train-mails
dictionary = make_Dictionary('ling-spam/train-mails')
dictionary


[('order', 1414),
 ('address', 1293),
 ('report', 1216),
 ('mail', 1127),
 ('send', 1079),
 ('language', 1072),
 ('email', 1051),
 ('program', 1001),
 ('our', 987),
 ('list', 935),
 ('one', 917),
 ('name', 878),
 ('receive', 826),
 ('money', 788),
 ('free', 762),
 ('work', 755),
 ('information', 677),
 ('business', 654),
 ('please', 652),
 ('university', 595),
 ('us', 564),
 ('day', 556),
 ('follow', 544),
 ('internet', 520),
 ('over', 511),
 ('http', 479),
 ('check', 472),
 ('call', 469),
 ('each', 466),
 ('include', 452),
 ('com', 448),
 ('linguistic', 442),
 ('number', 423),
 ('want', 420),
 ('letter', 419),
 ('need', 418),
 ('many', 412),
 ('here', 397),
 ('market', 395),
 ('start', 390),
 ('even', 386),
 ('fax', 383),
 ('form', 380),
 ('most', 377),
 ('first', 373),
 ('web', 366),
 ('service', 363),
 ('interest', 362),
 ('software', 352),
 ('remove', 349),
 ('read', 347),
 ('those', 345),
 ('week', 344),
 ('every', 332),
 ('credit', 329),
 ('ll', 326),
 ('site', 320),
 ('much', 31

### Feature Extraction Process
Once the dictionary is ready, we can extract word count vector (our feature here) of 3000 dimensions for each email of the training set. Each **word count vector** contains the frequency of 3000 words in the training file. Of course you might have guessed by now that most of them will be zero. Let us take an example. Suppose we have 500 words in our dictionary. Each word count vector contains the frequency of 500 dictionary words in the training file. Suppose the text in the training file is “Get the work done, work done”, then it will be encoded as $$[0,0,0,0,0,…….0,0,2,0,0,0,……,0,0,1,0,0,…0,0,1,0,0,……2,0,0,0,0,0]$$ Here, all the word counts are placed at the 296th, 359th, 415th, 495th elements of the word count vector in the length of 500 and the rest are zero.

The below python code will generate a feature vector matrix whose rows denote 700 files of the training set and columns denote 3000 words of the dictionary. The value at index ${ij}$ will be the number of occurrences of the $j^{th}$ word of the dictionary in the $i^{th}$ file

In [3]:
def extract_features(mail_dir):
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files),3000))
    docID = 0
    _i = 0
    print(len(files))
    for fil in files:
        _i+=1
        with open(fil) as fi:
            for i,line in enumerate(fi):
                if i == 2:
                    words = line.split()
                    for word in words:
                        wordID = 0
                        for i,d in enumerate(dictionary):
                            if d[0] == word:
                                wordID = i
                                features_matrix[docID,wordID]+=1
            docID = docID + 1
        print('\r','done {} files'.format(_i),flush=True,end='')
    return features_matrix

In [4]:
features_matrix = extract_features("ling-spam/train-mails")
features_matrix

702
 done 3 files

 done 702 files

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

### Training the Classifiers
Here you should write your Naïve Bayes classifiers after fully understanding its principle.

In [5]:
print(features_matrix.shape)

(702, 3000)


In [41]:
########### Write Your Code Here ###########
msg = []
spm = []
for i in range(len(features_matrix)):
    if i < 350:
        msg.append(features_matrix[i])
    else:
        spm.append(features_matrix[i])
msg = np.array(msg)
spm = np.array(spm)
msg_mean = []
msg_var = []
spm_mean = []
spm_var = []
for i in range(msg.shape[1]):
    msg_mean.append(np.mean(msg[:,i]))
    tmp_var = np.var(msg[:,i])
    if tmp_var == 0:
        tmp_var = 0.01
    msg_var.append(tmp_var)
for i in range(spm.shape[1]):
    spm_mean.append(np.mean(spm[:,i]))
    tmp_var = np.var(spm[:,i])
    if tmp_var == 0:
        tmp_var = 0.01
    spm_var.append(tmp_var)

print(msg_mean)
print(msg_var)
print(spm_mean)
print(spm_var)
############################################

0.053317142857142855
[0.3171428571428571, 0.47714285714285715, 0.11714285714285715, 0.20857142857142857, 0.46, 3.0485714285714285, 0.1657142857142857, 0.32, 0.3, 0.43142857142857144, 0.9514285714285714, 0.32857142857142857, 0.19142857142857142, 0.04857142857142857, 0.08, 0.5285714285714286, 0.5971428571428572, 0.037142857142857144, 0.4742857142857143, 1.6542857142857144, 0.46285714285714286, 0.20285714285714285, 0.46285714285714286, 0.05714285714285714, 0.19714285714285715, 0.3, 0.08857142857142856, 0.34, 0.17714285714285713, 0.4142857142857143, 0.20857142857142857, 1.2542857142857142, 0.3742857142857143, 0.19142857142857142, 0.13142857142857142, 0.23714285714285716, 0.36, 0.3485714285714286, 0.03428571428571429, 0.07714285714285714, 0.29714285714285715, 0.31142857142857144, 0.5571428571428572, 0.32, 0.28, 0.12, 0.11428571428571428, 0.5628571428571428, 0.08857142857142856, 0.03428571428571429, 0.20857142857142857, 0.3142857142857143, 0.11428571428571428, 0.07428571428571429, 0.04285714

### Checking Performance
The test set contains 130 spam emails and 130 non-spam emails. Please compute accuracy, recall, F-1 score to evaluate the performance of your spam filter.

In [9]:
test_features_matrix = extract_features("ling-spam/test-mails")
test_features_matrix

260
 done 260 files

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [17.,  2.,  0., ...,  0.,  0.,  0.]])

In [37]:
predict_label = []
for i in range(test_features_matrix.shape[0]):
    msg_prob = 0
    spm_prob = 0
    for j in range(test_features_matrix.shape[1]):
        msg_prob += np.log(1/(np.sqrt(2*np.pi*msg_var[j]))*np.exp(-(test_features_matrix[i,j]-msg_mean[j])**2/(2*msg_var[j])))
        spm_prob += np.log(1/(np.sqrt(2*np.pi*spm_var[j]))*np.exp(-(test_features_matrix[i,j]-spm_mean[j])**2/(2*spm_var[j])))
    if msg_prob > spm_prob:
        predict_label.append(0) #msg
    else:
        predict_label.append(1) #spm
    
# calculate the accuracy
count = 0
for i in range(130):
    if predict_label[i] == 0:
        count += 1
for i in range(130,260):
    if predict_label[i] == 1:
        count += 1
accuracy = count/260
print("accuracy:", accuracy)
# calculate the recall
count = 0
for i in range(130):
    if predict_label[i] == 0:
        count += 1
recall = count/130
# calculate the F-1 score
f1 = 2*accuracy*recall/(accuracy+recall)
print("F-1 score:", f1)

    

C:\Users\DELL\AppData\Local\Temp\ipykernel_20512\3886251665.py:7: RuntimeWarning: divide by zero encountered in log
  spm_prob += np.log(1/(np.sqrt(2*np.pi*spm_var[j]))*np.exp(-(test_features_matrix[i,j]-spm_mean[j])**2/(2*spm_var[j])))
C:\Users\DELL\AppData\Local\Temp\ipykernel_20512\3886251665.py:6: RuntimeWarning: divide by zero encountered in log
  msg_prob += np.log(1/(np.sqrt(2*np.pi*msg_var[j]))*np.exp(-(test_features_matrix[i,j]-msg_mean[j])**2/(2*msg_var[j])))


accuracy: 0.8384615384615385
recall: 0.2846153846153846
F-1 score: 0.4249736564805058


### Questions
1. Describe another real-world application where the naïve Bayes method can be applied
2. What are the strengths of the naïve Bayes method; when does it perform well?
3. What are the weaknesses of the naïve Bayes method; when does it perform poorly?
4. What makes the naïve Bayes method a good candidate for the classification problem, if you have enough knowledge about the data?

In [47]:
########### Write Your Code Here ###########
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
gnb = GaussianNB()
gnb.fit(features_matrix, np.array([0]*351+[1]*351))
y_pred = gnb.predict(test_features_matrix)
accuracy = accuracy_score(np.array([0]*130+[1]*130), y_pred)
print("accuracy:", accuracy)
recall = accuracy_score(np.array([0]*130), y_pred[:130])
print("recall:", recall)
f1 = 2*accuracy*recall/(accuracy+recall)
print("F-1 score:", f1)
############################################

accuracy: 0.9615384615384616
recall: 0.9923076923076923
F-1 score: 0.9766807995154452
